In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from typing import *

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', None)
pd.set_option('future.no_silent_downcasting', True)
sns.set()

df = pd.read_csv('data_part_2.csv')

print(df.shape)
df.head()

(1009, 18)


,user_id,region,device,channel,session_start,session_end,sessiondurationsec,session_date,month,day,hour_of_day,order_dt,revenue,payment_type,promo_code,final_price,time_of_day,sold
0,529697267522,United States,iPhone,социальные сети,2019-05-01 00:06:40,2019-05-01 00:07:06,26.0,2019-05-01,5,3,0,2019-05-01 00:06:40,9999.0,Mobile payments,0.0,9999.0,night,1
1,601292388085,United States,PC,organic,2019-05-01 06:56:16,2019-05-01 07:09:18,782.0,2019-05-01,5,3,7,NaN,NaN,NaN,NaN,NaN,morning,0
2,852898876338,United States,Mac,социальные сети,2019-05-01 04:30:45,2019-05-01 04:34:56,251.0,2019-05-01,5,3,4,NaN,NaN,NaN,NaN,NaN,night,0
3,998513020664,United States,iPhone,социальные сети,2019-05-01 18:53:42,2019-05-01 18:57:35,233.0,2019-05-01,5,3,18,NaN,NaN,NaN,NaN,NaN,evening,0
4,240702200943,United States,Mac,социальные сети,2019-05-02 14:04:32,2019-05-02 14:09:51,319.0,2019-05-02,5,4,14,NaN,NaN,NaN,NaN,NaN,day,0


# Set Proper Data Types

In [2]:
dtype_mapping = {
    'user_id': 'int64', 
    'region': 'object', 
    'device': 'object', 
    'channel': 'object', 
    'session_start': 'datetime64[ns]', 
    'session_end': 'datetime64[ns]', 
    'sessiondurationsec': 'float64', 
    'session_date': 'datetime64[ns]', 
    'month': 'int64', 
    'day': 'int64', 
    'hour_of_day': 'int64', 
    'order_dt': 'datetime64[ns]', 
    'revenue': 'float64', 
    'payment_type': 'object', 
    'promo_code': 'float64'
}

df = df.astype(dtype_mapping)

# Custom Functions

# 1. Data Preparation:

In [6]:
def prepare_daily_device_purchases(data):
    daily_purchases = (data
                      .groupby(['session_date', 'region', 'device'])['sold']
                      .sum()
                      .reset_index())
    return daily_purchases

# Let's look at our data structure
daily_data = prepare_daily_device_purchases(df)
print("Sample of daily data:")
print(daily_data.head())

Sample of daily data:
  session_date         region   device  sold
0   2019-05-01  United States      Mac     0
1   2019-05-01  United States       PC     0
2   2019-05-01  United States   iPhone     1
3   2019-05-02         France      Mac     0
4   2019-05-02  United States  Android     0


# 2. Check for normality:

In [7]:
def test_normality_device_purchases(daily_data):
    # Shapiro-Wilk test for each combination
    results = []
    for region in daily_data['region'].unique():
        for device in daily_data['device'].unique():
            mask = (daily_data['region'] == region) & (daily_data['device'] == device)
            purchases = daily_data[mask]['sold']
            
            stat, pval = stats.shapiro(purchases)
            results.append({
                'region': region,
                'device': device,
                'n_samples': len(purchases),
                'shapiro_p': pval
            })
    
    return pd.DataFrame(results)

normality_results = test_normality_device_purchases(daily_data)
print("\nNormality test results:")
print(normality_results)


Normality test results:
           region   device  n_samples     shapiro_p
0   United States      Mac        108  2.583712e-14
1   United States       PC         65  1.958821e-11
2   United States   iPhone        147  2.656300e-15
3   United States  Android        101  6.077111e-13
4          France      Mac         14  9.202771e-05
5          France       PC         38  1.302243e-09
6          France   iPhone         20  1.846016e-08
7          France  Android         29  3.601117e-08
8         Germany      Mac         14  7.890862e-06
9         Germany       PC         34  1.202452e-09
10        Germany   iPhone         23  1.034132e-06
11        Germany  Android         24  3.156269e-07
12             UK      Mac         13  1.159926e-04
13             UK       PC         45  1.676103e-10
14             UK   iPhone         19  3.687400e-05
15             UK  Android         28  1.124699e-07


# Since all p-values < 0.05 for normality tests, we should use non-parametric test (Kruskal-Wallis) to test device effect in each region.

In [8]:
def test_device_effect_by_region(daily_data):
    results = []
    
    for region in daily_data['region'].unique():
        region_data = daily_data[daily_data['region'] == region]
        
        # Get purchase data for each device
        device_groups = [group['sold'].values 
                        for _, group in region_data.groupby('device')]
        
        # Kruskal-Wallis test
        stat, pval = stats.kruskal(*device_groups)
        
        # Get summary statistics
        device_stats = region_data.groupby('device')['sold'].agg(['size', 'mean'])
        
        results.append({
            'region': region,
            'statistic': stat,
            'p_value': pval,
            'significant': pval < 0.05,
            **{f'{dev}_n': device_stats.loc[dev, 'size'] 
               for dev in device_stats.index},
            **{f'{dev}_mean': device_stats.loc[dev, 'mean'] 
               for dev in device_stats.index}
        })
    
    return pd.DataFrame(results)

# Run the test
test_results = test_device_effect_by_region(daily_data)
print("Test results by region:")
print(test_results)

Test results by region:
          region  statistic   p_value  significant  Android_n  Mac_n  PC_n  iPhone_n  Android_mean  Mac_mean   PC_mean  iPhone_mean
0  United States   3.423439  0.330828        False        101    108    65       147      0.485149  0.435185  0.353846     0.530612
1         France   4.748734  0.191148        False         29     14    38        20      0.275862  0.428571  0.263158     0.100000
2        Germany   1.680685  0.641236        False         24     14    34        23      0.291667  0.214286  0.205882     0.347826
3             UK   1.177001  0.758526        False         28     13    45        19      0.321429  0.384615  0.266667     0.421053


# No significant differences found. Let's combine devices into mobile (iPhone + Android) and desktop (Mac + PC) categories, as this grouping is logical and might reveal patterns that are hidden when devices are split too granularly

In [9]:
def test_mobile_desktop_effect_by_region(data):
    # First combine devices into mobile/desktop
    data = data.copy()
    data['device_type'] = data['device'].map({
        'iPhone': 'mobile',
        'Android': 'mobile',
        'Mac': 'desktop',
        'PC': 'desktop'
    })
    
    # Aggregate to daily level
    daily_data = (data.groupby(['session_date', 'region', 'device_type'])['sold']
                 .sum()
                 .reset_index())
    
    results = []
    for region in daily_data['region'].unique():
        region_data = daily_data[daily_data['region'] == region]
        
        # Get mobile and desktop data
        mobile_data = region_data[region_data['device_type'] == 'mobile']['sold']
        desktop_data = region_data[region_data['device_type'] == 'desktop']['sold']
        
        # Mann-Whitney U test (for two groups)
        stat, pval = stats.mannwhitneyu(mobile_data, desktop_data, alternative='two-sided')
        
        results.append({
            'region': region,
            'statistic': stat,
            'p_value': pval,
            'significant': pval < 0.05,
            'mobile_n': len(mobile_data),
            'desktop_n': len(desktop_data),
            'mobile_mean': mobile_data.mean(),
            'desktop_mean': desktop_data.mean()
        })
    
    return pd.DataFrame(results)

# Run the test
combined_results = test_mobile_desktop_effect_by_region(df)
print("\nTest results after combining devices:")
print(combined_results)


Test results after combining devices:
          region  statistic   p_value  significant  mobile_n  desktop_n  mobile_mean  desktop_mean
0  United States    12650.5  0.029154         True       168        133     0.755952      0.526316
1         France     1012.5  0.492671        False        45         48     0.222222      0.333333
2        Germany     1081.5  0.331919        False        46         43     0.326087      0.232558
3             UK     1165.0  0.720463        False        45         50     0.377778      0.340000


# Based on our statistical analysis of whether device type influences daily purchase rates across regions, we can conclude:

1. United States Market:
- A significant difference was found between mobile and desktop devices (p = 0.029 < 0.05)
- Mobile devices show higher average daily purchases (0.76) compared to desktop devices (0.53)
- The sample sizes are robust (mobile n=168, desktop n=133)

2. Other Regional Markets:
- No significant differences were found between device types in:
  * France (p = 0.493)
  * Germany (p = 0.332)
  * UK (p = 0.720)
- However, smaller sample sizes in these regions (n≈45-50) may limit the power to detect differences

Answer to Research Question:
Device type does influence daily purchase rates, but this effect is market-dependent. The influence is statistically significant only in the United States market, where mobile devices demonstrate higher purchase activity. The lack of significant findings in other regions might be due to smaller sample sizes or genuinely different market dynamics.

Limitations:
1. Uneven market representation with US having considerably more data
2. Relatively small sample sizes in non-US markets
3. Results reflect specific time period of data collection

These findings suggest that device-specific marketing strategies might be particularly relevant for the US market, where mobile platforms show stronger performance.